**I asked a question on [math.stackexchange.com](math.stackexchange.com), and recieved a great answer. This is my attempt to implement the insighful solution** ([link to question](http://math.stackexchange.com/questions/1350456/probability-a-blackjack-dealer-will-bust-if-you-know-their-score-and-know-the-ex))

**After some thought and reading comments on math stackexchange, i decided to implement the recursive solution rather than the linear algebra one for now**

In [6]:
from __future__ import division
import numpy as np
from copy import deepcopy

In [21]:
class Shoe:
    
    def __init__(self, state):
        '''
        deck state is a vector of the number of cards
        of a specific value in order from ace to 10
        '''
        self.state = state

    def p_busts(self, hand, p=0):
        for card in range(1,11):
            if hand.score <= 16:
                score_after_hitting = hand.hit(card)[0]
                if score_after_hitting >= 22:
                    p += sum(self.state.values()[card - 1:]) / sum(self.state.values())
                    break
                elif score_after_hitting <= 16:
                    new_state = deepcopy(self.state)
                    new_state[card] -= 1
                    p += Shoe(new_state).p_busts(Hand(hand.hit(card)), p)
        return p

In [8]:
class Hand:
    
    def __init__(self, score, soft=False):
        self.score = score
        self.soft = False
        
    def hit(self, value):
        score = self.score
        soft = self.soft
        if soft:
            if score + value > 21:
                score = score - 10 + value
                soft = False
            else:
                score += value
        elif value == 1 and score + 11 <= 21:
            score += 11
            soft = True
        else:
            score += value
        return (score, soft)

In [22]:
default_state = {x:4*6 for x in range(1,10)}
default_state[10] = 4*4*6
shoe = Shoe(default_state)
p = 0
for x in range(1,11):
    new_state = deepcopy(shoe.state)
    new_state[x] -= 1
    the_shoe = Shoe(new_state)
    if x == 0:
        p += the_shoe.p_busts(Hand(17, soft=True))*the_shoe.state[x]/sum(the_shoe.state)
    else:
        p += the_shoe.p_busts(Hand(6+x))*the_shoe.state[x]/sum(the_shoe.state)
print p


0.308681672026
0.385852090032
0.459807073955
0.536977491961
0.614147909968
1.76807950892


In [20]:
default_state.values()[5:]

[24, 24, 24, 24, 96]

----------
<blockquote>
I think this question is related to a broader question that I ask myself all the time in research: what does it mean to have a "nice solution" to a problem? When I was young, I was taught that a "nice solution" is a formula for the thing you want to calculate, but that's not always true! Having a forumla for something often tells you very little about it, and other descriptions are often much more useful from a practical point of view.

I'm not sure whether the description of the bust probability given above is much use, but for this problem, I suspect that a linear algebraic description of this kind will be more useful than a formula.
</blockquote>